<span style="font-size: 4em;">Welcome to PyTreeMotif</span>
<p></p>

#### an enumerative motif finding algorithm that makes trees

# IMPORTS 

Dependencies, can't live with em, can't live without em

In [1]:
from collections import deque
import copy
import random
from multiprocessing import cpu_count, Pool
from tqdm import tqdm
import sys

 # CLASSES 
 
### Define custom classes needed for PyTreeMotif

### Sequence_Node

* nodes representing lmer sequences similar to a reference pair <p></p>

* these are candidates from which tree nodes will be selected <p></p>
 
* records sequence, position, and the input sequence in which it was found <p></p>

In [2]:
class Sequence_Node:
    def __init__(self, seq, id, pos):
        self.seq = seq
        self.id = id
        self.pos = pos

### Sequence_Node_Set

* a set of Sequence_Nodes from one input sequence that are similar to a reference pair


In [3]:
class Sequence_Node_Set:
    def __init__(self, id):
        self.id = id
        self.members = set()
        
    def add_member(self, member):
        self.members.add(member)

### Reference_Pair

* reference pair of lmer sequences differing by at most 2d <p></p>

* one is from input sequence 0, the other from input sequence 1 <p></p>

* reference pairs are the basis on which all other possible nodes are selected <p></p>

* contains a pair of sequences, their positions, and an id number for the pair <p></p>

* contains a list of Sequence_Node_Sets, one for each of the remaining input sequences <p></p>

* after tree construction, contains a list of trees built from this reference pair <p></p>

In [4]:
class Reference_Pair:
    def __init__(self, pair_id, seq1, seq2, pos1, pos2, num_seq):
        self.pair_id = pair_id
        self.seq1 = seq1
        self.seq2 = seq2
        self.pos1 = pos1
        self.pos2 = pos2
        
        self.node_sets = [] 
        for i in range(2, num_seq):
            self.node_sets.append(Sequence_Node_Set(i))
            
        self.tree_list = []

        
    

### Tree_Node

* nodes in a motif tree represent lmer sequences which are at most 2d away from all other nodes in the path to the root <p></p>
* contains lmer sequence, position, input sequence origin, depth, children, and parent <p></p>
* child nodes can be added and removed <p></p>

In [5]:
class Tree_Node:
    def __init__(self, seq, seq_id, pos, depth=0):
        self.seq = seq
        self.seq_id = seq_id
        self.pos = pos
        self.depth = depth
        
        self.children = []
        self.parent = None

    def add_child(self, child_node):
        child_node.parent = self
        child_node.depth = self.depth + 1
        self.children.append(child_node)
        

    def remove_child(self, child_node):
        if child_node in self.children:
            child_node.parent = None
            self.children.remove(child_node)

### Tree

* constructed trees represent one or more cliques of motifs <p></p>
* one path from root to leaf represents a clique of motif instances <p></p>
* trees with multiple paths contains multiple cliques <p></p><p></p>
* the root node is a tree node with lmer at most 2d from a reference pair <p></p>
* all root nodes are from sequence 2 (reference pairs are from sequences 0 and 1) <p></p>
* each level of the tree represents nodes from each input sequence (besides the ref seqs) <p></p>
* leaves are stored for to be easily checked during branch pruning <p></p>

In [6]:
class Tree:
    def __init__(self, root):
        self.root = root
        self.current_leaves = []
        self.height = 0
        self.id = 0
    
    def save_leaf(self, leaf):
        self.current_leaves.append(leaf)
        
    def unsave_leaf(self, leaf):
        if leaf in self.current_leaves:
            self.current_leaves.remove(leaf)

### Clique

* a set of instances of a motif which are all at most 2d different <p></p>
* a clique is initially formed by finding a path through a tree from root to leaf <p></p>
* cliques can be merged if all lmers in one are at most 2d from all lmers in another <p></p>
* merged cliques will then contain more instances than the number of input sequences <p></p>
* for dedicated testing inputs where there is deliberately one motif per input sequence, merging may pick up random instances only similar by chance <p></p>
* when number of occurrences of motif per input is unknown, this may help detect multiple motif instances in the same input sequence, but other modifications to the algorithm are required in that scenario <p></p>

In [7]:
class Clique:
    def __init__(self, path):
        self.nodes = path
        self.seqs = []
        for node in self.nodes:
            self.seqs.append(node.seq)
            
        self.consensus = ""
        self.score = -1
        self.rank = -1
        
        self.merged = False
        
    def __deepcopy__(self, memo):
        new_nodes = []
        new_seqs = []
        for node in self.nodes:
            new_nodes.append(Tree_Node(node.seq, node.seq_id, node.pos))
            new_seqs.append(node.seq)
        new_copy = Clique(new_nodes)
        new_copy.seqs = new_seqs
        return new_copy
            

# FUNCTIONS #

### Define subroutines to be used by the main steps of PyTreeMotif ###

### read_file_to_list 

* reads a text file of one input sequence per line into a python list of input sequences


In [8]:
def read_file_to_list(filename):
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
        lines = [line.strip() for line in lines]
        
        return lines
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []

### hamming_distance

* measures hamming distance, the number of single character differences, between two strings


In [9]:
def hamming_distance(str1, str2):

    return sum(bit1 != bit2 for bit1, bit2 in zip(str1, str2))
    

### extendable

* given a tree and candidate node, finds all possible leaves to which the node could be added <p></p>
* breadth-first traversal of the tree, adding all children that are similar to node to queue <p></p>
* if the queue is empty, then the node could not be added to any leaves <p></p>
* traversal ends after finishing second to lowest level <p></p>
* all nodes left in the queue are the leaves of branches where every node in the branch all the way to the root is at most 2d from the new node <p></p>

In [10]:
def extendable(node, tree, ith_seq, _2d):
    queue = deque()
    
    # node must first be similar to root 
    if hamming_distance(tree.root.seq, node.seq) <= _2d:
        queue.append(tree.root)
        
        # while there are still potential branches
        while len(queue) > 0 and queue[0].depth < ith_seq - 3:
            front = queue.popleft()
            
            # check if node is similar to any child nodes
            for child in front.children:
                if hamming_distance(child.seq, node.seq) <= _2d:
                    queue.append(child)
    return queue
        

### prune_branches

* blah blah

In [11]:
def prune_branches(tree):
    for leaf in tree.current_leaves:
        if leaf.depth < tree.height:
            current_node = leaf
            num_siblings = len(current_node.parent.children) - 1
            branch_list = [leaf.seq]
            while num_siblings == 0:
                current_node = current_node.parent
                num_siblings = len(current_node.parent.children) - 1
                branch_list.append(current_node.seq)
            parent = current_node.parent
            parent.remove_child(current_node)
            tree.unsave_leaf(leaf)
            

### find_paths

* blah blah

In [12]:
def find_paths(node, path, paths):
    if node is None:
        return
    path.append(node)
    if not node.children:
        clique = Clique(path)
        paths.append(clique)
    
    for child in node.children:
        # branch_path = copy.deepcopy(path)
        find_paths(child, path[:], paths)
        
    
    
    return paths

### merge

* blah blah

In [13]:
def merge(new_cliques, _2d):
    
    # for prev_clique in merged_cliques:
    #     
    #     match = True
    #     for seq_1 in prev_clique.seqs:
    #         if not match:
    #             break
    #         for seq_2 in new_clique.seqs:
    #             if hamming_distance(seq_1, seq_2) > _2d:
    #                 match = False
    #                 break
    # 
    #     if match:
    #         merged_clique = copy.deepcopy(prev_clique)
    #         for node in new_clique.nodes:
    #             if node.seq not in prev_clique.seqs:
    #                 copy_node = Tree_Node(node.seq, node.seq_id, node.pos)
    #                 merged_clique.nodes.append(copy_node)
    #                 merged_clique.seqs.append(node.seq)
    #         new_cliques.append(merged_clique)
    # new_cliques.append(new_clique)
    # merged_cliques.extend(new_cliques)

SyntaxError: incomplete input (3345616950.py, line 23)

### score_motifs

* blah blah

In [14]:
def score_motifs(motifs, motif_length):
    scores = []
    for motif in motifs:
        
        profile = [[0] * motif_length for _ in range(4)]
        indices = {
            'A': 0,
            'C': 1,
            'G': 2,
            'T': 3,
            0: 'A',
            1: 'C',
            2: 'G',
            3: 'T'
        }
        for i in range(len(motif.seqs)):
            for j in range(motif_length):
                profile[indices[motif.seqs[i][j]]][j] += 1
                
        consensus = ""
                
        for j in range(motif_length):
            max_base = max(row[j] for row in profile)
            
            max_base_index = next(idx for idx, row in enumerate(profile) if row[j] == max_base)
            consensus += indices[max_base_index]
            
        score = 0
        for seq in motif.seqs:
            score += hamming_distance(seq, consensus)
        
        motif.consensus = consensus
        motif.score = score
        scores.append((score, motif))
    
    sorted_scores = sorted(scores, key=lambda x: x[0])
    
    rank = 1
    sorted_scores[0][1].rank = rank

    for i in range(1, len(sorted_scores)):
        if sorted_scores[i][0] > sorted_scores[i - 1][0]:
            rank += 1
        sorted_scores[i][1].rank = rank        
            
        

# STEP 1: NODE SELECTION #

### Find reference pairs from the reference sequences. 

### Then, find lmers similar to the reference pair from all other input sequences

### If a reference pair can't find at least one similar lmer from each input sequence, its not valid



In [15]:
def select_nodes(sequence_list, l, _2d):
        
    num_seq = len(sequence_list)
        
    reference_seq_1 = sequence_list[0]
    reference_seq_2 = sequence_list[1]
    reference_pair_set = set()
    reference_pair_list = []
    reference_pair_counter = 0
        
    for i in tqdm(range(len(reference_seq_1) - l + 1), desc='Finding reference pairs and sequence nodes', position=0, leave=True, file=sys.stdout):
        for j in range(len(reference_seq_2) - l + 1):
            if hamming_distance(reference_seq_1[i:i+l], reference_seq_2[j:j+l]) <= _2d:
                ref_sub_seq = reference_seq_1[i:i+l]
                ref_sub_seq2 = reference_seq_2[j:j+l]
                if (ref_sub_seq, ref_sub_seq2) in reference_pair_set:
                    break
                reference_pair = Reference_Pair(pair_id=reference_pair_counter, seq1=ref_sub_seq, seq2=ref_sub_seq2, pos1=i, pos2=j, num_seq=num_seq)    
                
                count_seqs_with_nodes = 0
                for k in range(2, num_seq):                
                    has_nodes = False
                    for m in range(len(sequence_list[k]) - l + 1):
                        sub_seq = sequence_list[k][m:m+l]
                        if hamming_distance(sub_seq, ref_sub_seq) <= _2d and hamming_distance(sub_seq, ref_sub_seq2) <= _2d:
                            seq_node = Sequence_Node(seq=sub_seq, id=k, pos=m)
                            reference_pair.node_sets[k-2].add_member(seq_node)
                            has_nodes = True
                    if has_nodes:
                        count_seqs_with_nodes += 1
                
                if count_seqs_with_nodes == num_seq - 2:
                    reference_pair_list.append(reference_pair)
                    reference_pair_set.add((ref_sub_seq, ref_sub_seq2))
                    reference_pair_counter += 1
                    
    return reference_pair_list
                

            




### TEST NODE SELECTION ###

# STEP 2: TREE CONSTRUCTION #

In [16]:

def construct_trees(reference_pair_list, _2d):
    
    num_seq = len(reference_pair_list[0].node_sets) + 2
    tree_count = 0
    cliques = []
    for pair in tqdm(reference_pair_list, desc='Building trees from reference pairs', position=0, leave=True, file=sys.stdout):
        node_sets = pair.node_sets
        for root in node_sets[0].members:
            root_node = Tree_Node(seq=root.seq, seq_id=root.id, pos=root.pos)
            tree = Tree(root_node)
            
            for i in range(3, num_seq):
                flag = False
                for node in node_sets[i-2].members:
                    branches = extendable(node, tree, i, _2d)
                    if branches:
                        # print("branches check")
                        for branch in branches:
                            new_leaf = Tree_Node(seq=node.seq, seq_id=node.id, pos=node.pos)
                            branch.add_child(new_leaf)
                            tree.save_leaf(new_leaf)
                            tree.unsave_leaf(branch)
                        flag = True
                if not flag:
                    tree = None
                    break
                tree.height += 1
                prune_branches(tree)
            if tree:    
                pair.tree_list.append(tree)
                tree.id = tree_count
                tree_count += 1
                
                new_cliques = find_paths(tree.root, [], [])

                for new_clique in new_cliques:
                    
                    new_clique.nodes.insert(0, Tree_Node(pair.seq2, 1, pair.pos2))
                    new_clique.seqs.insert(0, pair.seq2)
                    
                    new_clique.nodes.insert(0, Tree_Node(pair.seq1, 0, pair.pos1))
                    new_clique.seqs.insert(0, pair.seq1)
                    
                    cliques.append(new_clique)
                    
                    
    return cliques


### TEST TREE CONSTRUCTION ###

<span style="font-size: 5em;">PyTreeMotif</span>
<p></p>

### Define the entire algorithm as the function find_motif() ###

In [17]:
def find_motif(DNA, motif_length, d=-1, report_all=False, print_output=True):
    
    reference_pairs = None
    motifs = None
    _2d = 0
    
    if d == -1:
        i = 0
        while not reference_pairs and not motifs and i < 5:
            _2d = i * 2
            reference_pairs = select_nodes(DNA, motif_length, _2d)
            if not reference_pairs:
                i += 1
                continue
            motifs = construct_trees(reference_pairs, _2d)
            i += 1
    else:
        _2d = d * 2
        reference_pairs = select_nodes(DNA, motif_length, _2d)
        motifs = construct_trees(reference_pairs, _2d)
    
    if not reference_pairs or not motifs:
        return "no motifs found"
    
    score_motifs(motifs, motif_length)
    score_sorted_motifs = sorted(motifs, key=lambda motif: motif.rank)
    
    best_motifs = []
    i = 0
    while i < len(score_sorted_motifs) and score_sorted_motifs[i].rank == 1:
        best_motifs.append(score_sorted_motifs[i])
        i += 1
        
    
    if print_output:
        if report_all:
            print("ALL MOTIFS FOUND: ")
            for i in range(len(motifs)):
                print("Motif Rank: " + str(score_sorted_motifs[i].rank) + ", Consensus: " + score_sorted_motifs[i].consensus + ", Score: " + str(score_sorted_motifs[i].score))
                for node in score_sorted_motifs[i].nodes:
                    print("Seq: " + str(node.seq_id) + ", Motif Position: " + str(node.pos) + ", String: " + str(node.seq))
                print()
            
            print("Total Motifs: " + str(len(motifs)))
            print("-----------------------------------------------------------------------")
            
        for winner in best_motifs:
            print("Best Motif Consensus: " + str(winner.consensus) + ", Score: " + str(winner.score))
            print("Instances: ")
            for node in winner.nodes:
                print("Seq: " + str(node.seq_id) + ", Motif Position: " + str(node.pos) + ", String: " + str(node.seq))
            print()
            

            
            
    
    return [motif.consensus for motif in best_motifs]
    

# TEST CASES #

### First, test with the simple test cases provided on the course website ###

In [18]:
test1 = read_file_to_list("data1.txt")

find_motif(test1, 10)

Building trees from reference pairs: 100%|██████████| 1/1 [00:00<00:00, 4665.52it/s]
Best Motif Consensus: GGGTCTAAGC, Score: 0
Instances: 
Seq: 0, Motif Position: 67, String: GGGTCTAAGC
Seq: 1, Motif Position: 34, String: GGGTCTAAGC
Seq: 2, Motif Position: 12, String: GGGTCTAAGC
Seq: 3, Motif Position: 8, String: GGGTCTAAGC
Seq: 4, Motif Position: 14, String: GGGTCTAAGC
Seq: 5, Motif Position: 90, String: GGGTCTAAGC
Seq: 6, Motif Position: 51, String: GGGTCTAAGC
Seq: 7, Motif Position: 75, String: GGGTCTAAGC
Seq: 8, Motif Position: 56, String: GGGTCTAAGC
Seq: 9, Motif Position: 83, String: GGGTCTAAGC
Seq: 10, Motif Position: 72, String: GGGTCTAAGC
Seq: 11, Motif Position: 55, String: GGGTCTAAGC
Seq: 12, Motif Position: 34, String: GGGTCTAAGC
Seq: 13, Motif Position: 62, String: GGGTCTAAGC
Seq: 14, Motif Position: 44, String: GGGTCTAAGC



['GGGTCTAAGC']

In [19]:
test2 = read_file_to_list("data2.txt")

find_motif(test2, 10)

Building trees from reference pairs: 100%|██████████| 1/1 [00:00<00:00, 4832.15it/s]
Best Motif Consensus: GGGTCTAAGC, Score: 14
Instances: 
Seq: 0, Motif Position: 47, String: GGGTCTAAGG
Seq: 1, Motif Position: 41, String: GGGCCTAAGC
Seq: 2, Motif Position: 41, String: GGATCTAAGC
Seq: 3, Motif Position: 42, String: GGGTCTAACC
Seq: 4, Motif Position: 17, String: GGGTCTAAGC
Seq: 5, Motif Position: 60, String: GGGTCTAGGC
Seq: 6, Motif Position: 37, String: GGGTCGAAGC
Seq: 7, Motif Position: 18, String: GGGTCTAATC
Seq: 8, Motif Position: 43, String: GGGTCTGAGC
Seq: 9, Motif Position: 17, String: GGATCTAAGC
Seq: 10, Motif Position: 8, String: GGGTCAAAGC
Seq: 11, Motif Position: 83, String: AGGTCTAAGC
Seq: 12, Motif Position: 33, String: GGGTCGAAGC
Seq: 13, Motif Position: 20, String: GGGGCTAAGC
Seq: 14, Motif Position: 84, String: GGGTCCAAGC



['GGGTCTAAGC']

# 15,4 BENCHMARKING #
### Test the algorithm with the original subtle motif challenge proposed by Pevzner ###

First define the function to generate a single dataset.

It will take a sequence length argument.

First, a 15mer motif is randomly generated.

For each of 20 sequences:

1. The sequence is randomly generated at the given length
    
2. The motif is mutated 4 times at random positions with random bases
    
3. The mutated motif is implanted into the sequence at a random position

The dataset is now made up of 20 random sequences of the given length, each with a (15,4) implanted motif


In [20]:
def generate_15_4_dataset(seq_len):
    bases = ['A', 'C', 'G', 'T']
    
    motif = ''.join(random.choice(bases) for _ in range(15))
    
    # print("Implanted motif: " + motif)
    # print()
    
    dataset = []
    implanted_motifs = []
    
    for i in range(20):
        input_seq = ''.join(random.choice(bases) for _ in range(seq_len))
        
        sub_positions = [random.randint(0, 14) for _ in range(4)]
        
        implanted_motif = motif
        
        for position in sub_positions:
            mutation = random.choice(bases)
            implanted_motif = implanted_motif[:position] + mutation + implanted_motif[position + 1:]
            
        implant_position = random.randint(0, seq_len - 15)
        
        implanted_motifs.append((implanted_motif, implant_position))
            
        input_seq_w_motif = input_seq[:implant_position] + implanted_motif + input_seq[implant_position + 15:]
        
        dataset.append(input_seq_w_motif)
    
    # for im in implanted_motifs:
    #     print(im[0], im[1])
        
    return dataset, motif, implanted_motifs
        


### Test the 15,4 dataset generator ###

In [21]:
dataset, implanted_motif, instances = generate_15_4_dataset(200)
print("Implanted Motif: " + implanted_motif)
print("Instances: ")
for instance in instances:
    print(instance)

Implanted Motif: TATCCTCAGGATCGC
Instances: 
('GATCCTCAGGATCGT', 67)
('TTTCCTTAGGATCGC', 3)
('TATCCTCCGGATCGC', 111)
('TAACCTGAGGATCGT', 14)
('TCTCCCCAGGATCGT', 72)
('GAGCCTCAGGACCGC', 38)
('TACGCTCAGGATCGC', 104)
('TCTCTTCAGTATCGC', 54)
('TATATTCAGGATTGG', 161)
('TATCCTCAGGGACTC', 43)
('TATCGTCAAGATCGC', 100)
('TACCTTTAGGATCAC', 121)
('CATCGTTAGGATCGC', 83)
('TATCGCCAGGATCGC', 164)
('TTTCCCCAGCATCTC', 154)
('CATCCTCAGTATCGT', 118)
('TAGCCTCAGCATGAC', 135)
('TATCCTCAGGATCCA', 140)
('TATCCACAGGATCGC', 120)
('TATCCCAACGATCGC', 53)


### Test the algorithm on the above dataset ###

In [22]:
found_motif = find_motif(dataset, 15, 4, report_all=True)


Building trees from reference pairs: 100%|██████████| 104/104 [00:00<00:00, 10019.24it/s]
ALL MOTIFS FOUND: 
Motif Rank: 1, Consensus: TATCCTCAGGATCGC, Score: 54
Seq: 0, Motif Position: 67, String: GATCCTCAGGATCGT
Seq: 1, Motif Position: 3, String: TTTCCTTAGGATCGC
Seq: 2, Motif Position: 111, String: TATCCTCCGGATCGC
Seq: 3, Motif Position: 14, String: TAACCTGAGGATCGT
Seq: 4, Motif Position: 72, String: TCTCCCCAGGATCGT
Seq: 5, Motif Position: 38, String: GAGCCTCAGGACCGC
Seq: 6, Motif Position: 104, String: TACGCTCAGGATCGC
Seq: 7, Motif Position: 54, String: TCTCTTCAGTATCGC
Seq: 8, Motif Position: 161, String: TATATTCAGGATTGG
Seq: 9, Motif Position: 43, String: TATCCTCAGGGACTC
Seq: 10, Motif Position: 100, String: TATCGTCAAGATCGC
Seq: 11, Motif Position: 121, String: TACCTTTAGGATCAC
Seq: 12, Motif Position: 83, String: CATCGTTAGGATCGC
Seq: 13, Motif Position: 164, String: TATCGCCAGGATCGC
Seq: 14, Motif Position: 154, String: TTTCCCCAGCATCTC
Seq: 15, Motif Position: 118, String: CATCCTCAG

### Check the results ###

In [23]:
num_best_motifs = len(found_motif)
print(num_best_motifs)
for i in range(num_best_motifs):
    try:
        assert found_motif[i] == implanted_motif
        print("Success")
    except AssertionError:
        print("Fail")
        continue

1
Success


### Test with larger sequence length

In [31]:
dataset, implanted_motif, instances = generate_15_4_dataset(600)
large_set_found_motif = find_motif(dataset, 15, 4, report_all=True)


Building trees from reference pairs: 100%|██████████| 17299/17299 [00:05<00:00, 3083.98it/s]
ALL MOTIFS FOUND: 
Motif Rank: 1, Consensus: TCCGAACCGGTTGCG, Score: 49
Seq: 0, Motif Position: 431, String: GCCGAACTGGTTTCT
Seq: 1, Motif Position: 356, String: TTCGCACCTGTTTCG
Seq: 2, Motif Position: 411, String: TCCGGACCGGTTGCG
Seq: 3, Motif Position: 228, String: TCCCAACCGGTTGTG
Seq: 4, Motif Position: 546, String: CCCGTACTGGTTTCT
Seq: 5, Motif Position: 150, String: TTCGAGCCGCCTTCG
Seq: 6, Motif Position: 183, String: CCACAACCGGTTGCG
Seq: 7, Motif Position: 419, String: GACGTTCCGGATGTG
Seq: 8, Motif Position: 119, String: TCCGGACCGGTTGCG
Seq: 9, Motif Position: 274, String: TTCGCACATGTTGCG
Seq: 10, Motif Position: 3, String: TCCGAAGGGGTTGCG
Seq: 11, Motif Position: 130, String: TCCGAACCGATGGCG
Seq: 12, Motif Position: 45, String: TCCGATCCGGCTGCG
Seq: 13, Motif Position: 514, String: GCCGAATCGGCTGCG
Seq: 14, Motif Position: 330, String: TCCGAGCCTGTTTCG
Seq: 15, Motif Position: 480, String: 

In [32]:

for i in range(num_best_motifs):
    try:
        assert large_set_found_motif[i] == implanted_motif
        print("Success")
    except AssertionError:
        print("Fail")
        continue
print(implanted_motif)

Success
TCCGAACCGGTTGCG


### Test with 50 datasets ###

To replicate Sun et al's testing scheme as closely as possible, test the algorithm with 50 different datsets.

### NOTE: on a typical laptop this took _ minutes. 



In [26]:
implanted_motif_list = []
found_motif_list = []
for i in tqdm(range(50), desc='Testing datasets', position=0):
    dataset, implanted_motif, instances = generate_15_4_dataset(200)
    found_motif = find_motif(dataset, 15, 4, print_output=False)
    
    implanted_motif_list.append(implanted_motif)
    found_motif_list.append(found_motif)
    

Testing datasets:   0%|          | 0/50 [00:00<?, ?it/s]

Building trees from reference pairs: 100%|██████████| 178/178 [00:00<00:00, 15556.47it/s]

Testing datasets:   2%|▏         | 1/50 [00:11<09:35, 11.75s/it]


Building trees from reference pairs: 100%|██████████| 144/144 [00:00<00:00, 16631.69it/s]

Testing datasets:   4%|▍         | 2/50 [00:24<09:49, 12.29s/it]

...

Testing datasets:  96%|█████████▌| 48/50 [08:52<00:21, 10.98s/it]


Building trees from reference pairs: 100%|██████████| 127/127 [00:00<00:00, 5582.85it/s]

Testing datasets:  98%|█████████▊| 49/50 [09:03<00:10, 10.94s/it]


Building trees from reference pairs: 100%|██████████| 151/151 [00:00<00:00, 12139.69it/s]

Testing datasets: 100%|██████████| 50/50 [09:15<00:00, 11.10s/it]

See how many runs were successful before moving on to other metrics

In [29]:
correct_motifs = 0
for i in range(50):
    for motif in found_motif_list[i]:
        if motif == implanted_motif_list[i]:
            correct_motifs += 1
            break

In [30]:
print(str(correct_motifs) + "/50 motifs found")

50/50 motifs found


### PARALLELISM TESTING ###

In [ ]:
num_procs = cpu_count()
print(num_procs)

In [ ]:
def build_trees(pair, _2d):
    new_cliques = []
    
    
    node_sets = pair.node_sets
    print("Ref pair " + str(pair.pair_id) + ": " + str(len(node_sets[0].members)) + " roots")
    
    num_seq = len(node_sets) + 2
    for root in node_sets[0].members:
        root_node = Tree_Node(seq=root.seq, seq_id=root.id, pos=root.pos)
        tree = Tree(root_node)
        
        for i in range(3, num_seq):
            flag = False
            for node in node_sets[i-2].members:
                branches = extendable(node, tree, i, _2d)
                if branches:
                    # print("branches check")
                    for branch in branches:
                        new_leaf = Tree_Node(seq=node.seq, seq_id=node.id, pos=node.pos)
                        branch.add_child(new_leaf)
                        tree.save_leaf(new_leaf)
                        tree.unsave_leaf(branch)
                    flag = True
            if not flag:
                tree = None
                break
            tree.height += 1
            prune_branches(tree)
        if tree:    
            pair.tree_list.append(tree)
            # tree.id = tree_count
            # tree_count += 1
            
            new_cliques = find_paths(tree.root, [], [])

            for new_clique in new_cliques:
                
                new_clique.nodes.insert(0, Tree_Node(pair.seq2, 1, pair.pos2))
                new_clique.seqs.insert(0, pair.seq2)
                
                new_clique.nodes.insert(0, Tree_Node(pair.seq1, 0, pair.pos1))
                new_clique.seqs.insert(0, pair.seq1)     
                
    return new_cliques  

In [ ]:

def parallel_construct_trees(reference_pair_list, _2d):
    
    num_seq = len(reference_pair_list[0].node_sets) + 2
    unmerged_clique_lists = []
    merged_cliques = []
    
    
    num_processes = cpu_count()
    with Pool(num_processes) as pool:
        unmerged_clique_lists = pool.starmap(build_trees, [(pair, _2d) for pair in reference_pair_list])
    
    unmerged_cliques = [clique for sublist in unmerged_clique_lists for clique in sublist]
    
    for unmerged_clique in unmerged_cliques:
        merge(unmerged_clique, merged_cliques, _2d)


    return merged_cliques

In [ ]:
def parallel_find_motif(DNA, motif_length, d=1):
    
    _2d = d * 2
    
    reference_pairs = select_nodes(DNA, motif_length, _2d)
    
    if not reference_pairs:
        return "no reference pairs found"
    
    motifs = parallel_construct_trees(reference_pairs, _2d)
    
    score_motifs(motifs, motif_length)
    
    best_motif = None
    for motif in motifs:
        if motif.rank == 1:
            best_motif = motif
    
    print("Best Motif: Motif " + str(motifs.index(best_motif)+1) + ", " + str(best_motif.consensus))
    print()
        
    
    for i in range(len(motifs)):
        print("Motif " + str(i+1) + ", Consensus: " + motifs[i].consensus + ", Rank: " + str(motifs[i].rank) + ", Score: " + str(motifs[i].score))
        for node in motifs[i].nodes:
            print("Seq: " + str(node.seq_id) + ", Motif Position: " + str(node.pos) + ", String: " + str(node.seq))
        print()
        
    return best_motif.consensus

In [ ]:
parallel_found_motif = parallel_find_motif(dataset, 15, 4)
